# CC-cost for Sep/22

This script runs a `sql` code to create a panel data, based on the `case_records_current` table, where we have each `ICD-10` member change of status, and the link between the status change with change in daly weights. 
The calculation involves a lot of cleaning,   such as desconsidering impetous changes of status or *cancelled* icds status as the first status input, etc, we leave that for the sql code, and here is just the script to 
run it and export to the redshift.

Steps:

    - run this notebook to upload to Redshift;
    
    - use the new panel to make calculations, new tables, dashes...
***

## 0. Setup

In [2]:
#connection to redshift
from redshift_import_export.interfaces.data_interactor import DataInteractor
di = DataInteractor()
import requests


#standard packages
import warnings
warnings.filterwarnings('ignore')
import os
import collections
import pandas as pd
import numpy as np



#time packages
import datetime as dt

## 1. Import

In [28]:
jan_df = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/external/care-cost-historico - care_cost_jan_v6.csv')
fev_df = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/external/care-cost-historico - care_cost_fev_v6.csv')
mar_df = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/external/care-cost-historico - care_cost_mar_v6.csv')
apr_df = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/external/care-cost-historico - care_cost_abr_v6.csv')
may_df = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/external/care-cost-historico - care_cost_mai_v6.csv')
jun_df = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/external/care-cost-historico - care_cost_jun_v6.csv')
jul_df = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/external/care-cost-historico - care_cost_jul_v6.csv')
aug_df = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/external/care-cost-historico - care_cost_ago_v6.csv')
sep_df = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/external/care-cost-historico - care_cost_set_v6.csv')


## 2. Transform

In [29]:
jan_df_clean = jan_df.groupby('cid').sum()[['volume', 'care_cost_total']]
jan_df_clean['avg_care_cost_jan'] = jan_df_clean['care_cost_total']/jan_df_clean['volume']
#jan_df_clean['n_jan'] = 1
jan_df_clean = jan_df_clean[['avg_care_cost_jan']]
#

fev_df_clean = fev_df.groupby('cid').sum()[['volume', 'care_cost_total']]
fev_df_clean['avg_care_cost_fev'] = fev_df_clean['care_cost_total']/fev_df_clean['volume']
#fev_df_clean['n_fev'] = 1
fev_df_clean = fev_df_clean[['avg_care_cost_fev']]
#
mar_df_clean = mar_df.groupby('cid').sum()[['volume', 'care_cost_total']]
mar_df_clean['avg_care_cost_mar'] = mar_df_clean['care_cost_total']/mar_df_clean['volume']
#mar_df_clean['n_mar'] = 1
mar_df_clean = mar_df_clean[['avg_care_cost_mar']]
#
apr_df_clean = apr_df.groupby('cid').sum()[['volume', 'care_cost_total']]
apr_df_clean['avg_care_cost_apr'] = apr_df_clean['care_cost_total']/apr_df_clean['volume']
#apr_df_clean['n_apr'] = 1
apr_df_clean = apr_df_clean[['avg_care_cost_apr']]
#
may_df_clean = may_df.groupby('cid').sum()[['volume', 'care_cost_total']]
may_df_clean['avg_care_cost_may'] = may_df_clean['care_cost_total']/may_df_clean['volume']
#may_df_clean['n_may'] = 1
may_df_clean = may_df_clean[['avg_care_cost_may']]
#
jun_df_clean = jun_df.groupby('cid').sum()[['volume', 'care_cost_total']]
jun_df_clean['avg_care_cost_jun'] = jun_df_clean['care_cost_total']/jun_df_clean['volume']
#jun_df_clean['n_jun'] = 1
jun_df_clean = jun_df_clean[['avg_care_cost_jun']]
#
jul_df_clean = jul_df.groupby('cid').sum()[['volume', 'care_cost_total']]
jul_df_clean['avg_care_cost_jul'] = jul_df_clean['care_cost_total']/jul_df_clean['volume']
#jul_df_clean['n_jul'] = 1
jul_df_clean = jul_df_clean[['avg_care_cost_jul']]
#
aug_df_clean = aug_df.groupby('cid').sum()[['volume', 'care_cost_total']]
aug_df_clean['avg_care_cost_aug'] = aug_df_clean['care_cost_total']/aug_df_clean['volume']
#aug_df_clean['n_aug'] = 1
aug_df_clean = aug_df_clean[['avg_care_cost_aug']]
#
sep_df_clean = sep_df.groupby('cid').sum()[['volume', 'care_cost_total']]
sep_df_clean['avg_care_cost_sep'] = sep_df_clean['care_cost_total']/sep_df_clean['volume']
#sep_df_clean['n_sep'] = 1
sep_df_clean = sep_df_clean[['avg_care_cost_sep']]

In [30]:
df = jan_df_clean.merge(
    fev_df_clean, how='outer', left_index=True, right_index=True).merge(
        mar_df_clean, how='outer', left_index=True, right_index=True).merge(
            apr_df_clean, how='outer', left_index=True, right_index=True).merge(
                may_df_clean, how='outer', left_index=True, right_index=True).merge(
                    jun_df_clean,  how='outer', left_index=True, right_index=True).merge(
                        jul_df_clean, how='outer', left_index=True, right_index=True).merge(
                            aug_df_clean, how='outer', left_index=True, right_index=True).merge(
                                sep_df_clean, how='outer', left_index=True, right_index=True)




In [38]:
avg_care_cost_list = df.mean(axis = 1).rename('avg_care_cost').reset_index()
avg_care_cost_list.head(10)

,cid,avg_care_cost
0,A014,1098.743075
1,A02,59.211280
2,A045,661.298756
3,A05,205.927040
4,A059,926.130665
5,A06,464.673869
6,A071,266.428447
7,A08,859.667693
8,A080,278.126043
9,A082,0.000000


## 3. Export

In [39]:
di.redshift.insert_table(avg_care_cost_list, table_name='avg_care_cost_list', if_exists='replace')

Table avg_care_cost_list inserted successfully, with 2416 rows.
